### Iliti stabla odlucivanja

<img src='decision-tree.png'>

Stabla odlucivanja su jednostavan i interpretabilan model i koriste se i u klasifikacionim i u regresionim problemima masinskog ucenja. Sastoje se od grana i cvorova, gde krajnje cvorove zovemo listovima stabla. Funkcionisu tako sto:
* u svakom cvoru stabla, bez listova, imamo po jedan test kojim ispitujemo kriterijume za podelu stabla na osnovu atributa koje imamo u podacima
* svakom ishodu testa odgovara po jedna grana
* u listovima se nalaze vrednosti predikcija
* prednosti su sto odmah vidimo relacije izmedju atributa, kao i njihove znacajnosti prilikom podele podataka

### Kako treniramo?

* testovi koji se vrse u svakom od cvorova su tipicno provere vrednosti pojedinacnih atributa koje zavise od tipa modela
* koristi se pohlepni (greedy) algoritam optimizacije - heuristickim metodama trazi se najbolje trenutno resenje
* osnovna ideja je izabrati atribut koji najbolje razvrstava instance, tako da dobijene podgrupe budu sto homogenije u odnosu na klase u slucaju klasifikacije ili da prave sto manju gresku u slucaju regresije
* atribut se evaluira tako sto se izracuna smanjenje nehomogenosti nekon particionisanja skupa instanci po vrednostima tog atributa
* u toku treniranja modela ponavljamo proces rekurzivno sve dok se ne ispuni neki od zaustavnih kriterijuma, kao sto su, dosegnuta maksimalna dubina stabla, minimalni broj instanci u listu, dosegnuta vrednost mere homogonosti stabla itd <br> **Kompromis:** zaustavljanje i preprilagodjavanje (overfit)
* rezultat se tipicno izracunava kao prosek vrednosti u listu u slucaju regresije a u slucaju klasifikacije, kao vecinska klasa u listu

### Mere homogenosti (purity):

* entropy: $$ H(p_1, . . . , p_C ) = −\sum_{i=1}^{C}p_i*logp_i$$
* Gini index: $$ G(p_1, . . . , p_C ) = 1−\sum_{i=1}^{C}p_i^2$$

### Primer klasifikacionog stabla odlucivanja (CART)

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from sklearn import model_selection
from sklearn import metrics
from sklearn import preprocessing
from sklearn import datasets

In [ ]:
data = datasets.load_breast_cancer()

In [ ]:
print(data.DESCR)

In [ ]:
data.feature_names

In [ ]:
X = pd.DataFrame(data.data, columns = data.feature_names)

In [ ]:
X.shape

In [ ]:
X.info() # nema nedostajucih i cudnih tipova podataka

In [ ]:
X.describe()

In [ ]:
X.hist(figsize = [15,15]) 
plt.show()

In [ ]:
data.target_names

In [ ]:
y = data.target

In [ ]:
y

In [ ]:
print('Broj benignih instanci: ')
np.sum(y==1)

In [ ]:
print('Broj malignih instanci: ')
np.sum(y==0)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, stratify=y, random_state = 42)

In [ ]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

U paketu **tree** biblioteke `scikit-learn` nalaze se funkcije za rad sa stablima odlucivanja i njihovu vizuelizaciju.

In [ ]:
from sklearn import tree

Parametri:
*  `criterion` - kriterijum za odlucivanje o podeli (Gini indeks ili entropija) za problem klasifikacije
* `max_depth` - maksimalna dubina stabla
* `max_features` - maksimalni broj atributa koje nasumicno biramo prilikom podele minimalna vrednost čistoće grananja
* `min_samples_leaf` - minimalni broj instanci u listovima
* `random_state` - zbog reprodukcije eksperimenta jer postoje nasumicne odluke
* `min_impurity_split` - granicna vrednost heterogenosti ispod koje se zaustavlja podela

In [ ]:
model = tree.DecisionTreeClassifier(criterion='gini', max_features=0.9, max_depth=3, random_state=42)

`fit` i `predict` koristimo kao i do sada:

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

### Interpretacija

Pomocu funkcije `plot_tree` crtamo stablo odlucivanja:

In [ ]:
plt.figure(figsize=(20, 10))
tree.plot_tree(model, fontsize=12, feature_names=list(X.columns), filled=True, rounded=True, 
               class_names=['maligno','benigno'])

### Evaluacija

In [ ]:
metrics.accuracy_score(y_test, y_pred)

In [ ]:
metrics.confusion_matrix(y_test, y_pred)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
y_pred_proba = model.predict_proba(X_test)[::,1]

In [ ]:
y_pred_proba

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,  y_pred_proba) 

In [ ]:
auc = metrics.roc_auc_score(y_test, y_pred_proba) 

In [ ]:
auc

In [ ]:
plt.plot(fpr, tpr, label="AUC="+str(auc))
plt.ylabel('True Positive')
plt.xlabel('False Positive')
plt.legend(loc=4)
plt.show()

### Znacajnost atributa

Znacajnost svakog atributa izracunava se kao mera smanjenja heterogenosti (impurity) nakon podele instanci u cvoru na osnovu tog konkretnog atributa.

In [ ]:
plt.barh(list(X.columns), model.feature_importances_)
plt.show()